## 코드 통일

- 학습데이터는 [:1100]까지, 테스트데이터는 [1100:]로 하자. (추후 조정가능)

- 기본적으로 데이터프레임은 csv로 부터 불러온 것 중 학습데이터인 'df_learn'을 활용한다.  

- 지표이름이 'feature'라면, df의 행의 이름은 다음과 같이 한다.
>df_learn['PH_feature'], df_learn['PD_feature'], df_learn['PA_feature']

- 분석한 지표들의 지표이름은 다음과 같이한다. (분석될 때마다 추가)  

    1. IW    ;    북메이커 중 성공률이 가장 높은 IW의 배당률
    2. history    ; 팀의 전적.
        - PH = H승률 * A패율
        - PD = H무율 * A무율
        - PA = H패율 * A승률
    3. vs    ; 상대 전적

- 각 지표의 수치들은 [0,1]의 범위에서 표준화시킨다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df1718 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1718/E0.csv", sep=',', header = 0)
df1617 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1617/E0.csv", sep=',', header = 0)
df1516 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1516/E0.csv", sep=',', header = 0)
df = pd.concat([df1516,df1617,df1718], ignore_index = True)
df["HTW"] = (df["FTHG"] > df["FTAG"]).astype(int)
df["HTD"] = (df["FTHG"] == df["FTAG"]).astype(int)
df["HTL"] = (df["FTHG"] < df["FTAG"]).astype(int)


## 학습과 테스트 데이터 나누기

In [2]:
df_learn = df[:1100].copy()
df_test = df[1100:].copy()

## 배당률(IW)
df_learn['PH_IW']/D/L 를 추가한다. (기타 북메이커들도 있기는 함)

In [3]:
bookmakers = ['B365', 'IW', 'LB', 'PS', 'WH', 'VC']    # BW 는 918 행에 NaN이 들어가서 뺐다.
results = ['H', 'D', 'A']

for bookmaker in bookmakers:
    cases = [bookmaker+result for result in results]
    probs = df_learn[cases].apply(lambda x: 1/x)
    with_margin = probs.sum(axis=1)
    df_learn[['P'+case[-1]+'_'+case[:-1] for case in cases]] = probs.apply(lambda x: x/with_margin)

## 전적(history)
df_learn['PH_history']/D/L 를 추가한다.

In [32]:
match_data = ['HomeTeam', 'AwayTeam', 'HTW', 'HTD', 'HTL']
ht_prob = df_learn[match_data].groupby('HomeTeam')
ht_prob = ht_prob['HTW', 'HTD', 'HTL'].count() * ht_prob['HTW', 'HTD', 'HTL'].mean()
ht_prob.columns = ['W', 'D', 'L']
ht_prob.index.name = 'Team'
at_prob = df_learn[match_data].groupby('AwayTeam')
at_prob = at_prob['HTW', 'HTD', 'HTL'].count() * at_prob['HTW', 'HTD', 'HTL'].mean()
at_prob.columns = ['L', 'D', 'W']
at_prob.index.name = 'Team'

history_prob = pd.DataFrame(data=(ht_prob + at_prob), columns=['W', 'D', 'L'])
history_prob = history_prob.apply(lambda x: x/history_prob.sum(axis=1), axis=0)

hp_dict_w = history_prob['W'].T.to_dict()
hp_dict_d = history_prob['D'].T.to_dict()
hp_dict_l = history_prob['L'].T.to_dict()

# 전적 확률 식 바꾸고 싶으면 여기!!
df_learn['PH_history'] = df_learn['HomeTeam'].map(hp_dict_w)*df_learn['AwayTeam'].map(hp_dict_l)
df_learn['PD_history'] = df_learn['HomeTeam'].map(hp_dict_d)*df_learn['AwayTeam'].map(hp_dict_d)
df_learn['PA_history'] = df_learn['HomeTeam'].map(hp_dict_l)*df_learn['AwayTeam'].map(hp_dict_w)

df_learn.loc[:,'PH_history':'PA_history'] = df_learn.loc[:,'PH_history':'PA_history'].apply(lambda x: x/df_learn.loc[:,'PH_history':'PA_history'].sum(axis=1), axis=0)

## 상대전적(vs)
df_learn['PH_vs']/D/L 를 추가한다.

In [33]:
vs_prob = df_learn.groupby(['HomeTeam', 'AwayTeam']).mean().loc[:,'HTW':'HTL'].reset_index()
def vs_result(home, away, result):
    return vs_prob[(vs_prob['HomeTeam']==home) & (vs_prob['AwayTeam']==away)]['HT'+result].values[0]
df_learn['PH_vs']=df_learn.apply(lambda x: vs_result(x['HomeTeam'], x['AwayTeam'], 'W'), axis=1)
df_learn['PD_vs']=df_learn.apply(lambda x: vs_result(x['HomeTeam'], x['AwayTeam'], 'D'), axis=1)
df_learn['PA_vs']=df_learn.apply(lambda x: vs_result(x['HomeTeam'], x['AwayTeam'], 'L'), axis=1)

## 단일지표에 의한 예측 성공률 계산

In [37]:
def hit_rate_by(feature):
    '''
    하나의 지표에 의해서만 예측 성공률을 계산한다.
    df_learn에 feature 정보가 있어야 한다.
    '''
    results = ['H','D','A']
    cases = ['P'+result+'_'+feature for result in results]
    guess = df_learn[cases].idxmax(axis=1).map(lambda x: x[1])
    hit_table = (df_learn['FTR'] == guess).value_counts()
    return hit_table.loc[True] / hit_table.sum()

print('hit rate by IW', hit_rate_by('IW'))
print('hit rate by history', hit_rate_by('history'))
print('hit rate by vs', hit_rate_by('vs'))

hit rate by IW 0.5463636363636364
hit rate by history 0.5254545454545455
hit rate by vs 0.7454545454545455


## 다중지표와 가중치에 의한 예측 성공률 계산

In [40]:
def hit_rate_among(weights, features, to_print=False):
    '''
    가중치벡터(W)와 지표들(X)을 주면 예측성공률(z)를 계산한다.
    weights와 features는 list type
    features의 지표의 이름 string type의 list
    hit_rate_by()와 마찬가지로 df_learn[feature]가 이미 존재해야한다.
      ex) feature = 'IW' -> df_learn['PH_IW'] 준비
    '''
    results = ['H','D','A']
    wx_pair = list(zip(weights, features))
    prob_frame = pd.DataFrame(index=df_learn.index)
    for result in results:
        temp = pd.Series(0, index=df_learn.index)
        for wx in wx_pair:
            column_name = 'P'+result+'_'+wx[1]
            temp += wx[0]*df_learn[column_name]
        prob_frame[result] = temp
    
    guess = prob_frame[results].idxmax(axis=1)
    hit_table = (df_learn['FTR']==guess).value_counts()
    rate = hit_table.loc[True] / hit_table.sum()
    if to_print: print(weights, features, rate)
    return rate
    

## 다중지표에 의한 예측 성공률 테스트

In [ ]:
hit_rate_among([1],['IW'], True)
hit_rate_among([0.5, 0.5], ['IW', 'vs'], True)
weights = [
    [i/2, j/2, k/2] for i in range(3) for j in range(3) for k in range(3)
]
features = ['IW', 'history', 'vs']
for weight in weights:
    hit_rate_among(weight, features, True)

# 마지막 할일

## 로지스틱 회귀분석 적용

## 슛, 홈 지표 추가

## 테스트데이터로 실험해본 후 고찰
- 학습:테스트 양 조절
- hit_rate_among() VS 로지스틱 회귀분석